In [2]:
import sys
sys.path.insert(0, '../')

import utils
import pandas as pd
from tqdm import tqdm

import json


In [3]:
events_path = ['../../data/clean_data/events_2015_2018_filtered.csv', '../../data/clean_data/incidents.csv']
events_name = ['events', 'incidents']
json_path = '../../dashboards/data/json/'

In [7]:
for ep, file_name in tqdm(zip(events_path, events_name),
                          total=len(events_path)):

    events = pd.read_csv(ep).fillna('null')
    d = {}

    #list event per timestep
    df_ = events.set_index(['stop_id', 'start_datetime']).copy()
    for si, sd in df_.reset_index().drop_duplicates(['stop_id', 'start_datetime'])[['stop_id', 'start_datetime']].values:
        try:
            d[str(si)][sd]['list_event'] = df_.loc[si].loc[[sd]].to_dict(orient='records')
        except:
            try:
                d[str(si)][sd] = {}
                d[str(si)][sd]['list_event'] = df_.loc[si].loc[[sd]].to_dict(orient='records')
            except:
                d[str(si)] = {}
                d[str(si)][sd] = {}
                d[str(si)][sd]['list_event'] = df_.loc[si].loc[[sd]].to_dict(orient='records')
                
    # list event per day
    df_ = events.copy()
    df_['datetime'] = [i[:10] for i in df_['start_datetime'].values]
    df_ = df_.set_index(['stop_id', 'datetime'])
    for si, sd in df_.reset_index().drop_duplicates(['stop_id','datetime'])[['stop_id', 'datetime']].values:

        try:
            d[str(si)][sd]['list_event'] = df_.loc[si].loc[[sd]].to_dict(orient='records')
        except:
            try:
                d[str(si)][sd] = {}
                d[str(si)][sd]['list_event'] = df_.loc[si].loc[[sd]].to_dict(orient='records')
            except:
                d[str(si)] = {}
                d[str(si)][sd] = {}
                d[str(si)][sd]['list_event'] = df_.loc[si].loc[[sd]].to_dict(orient='records')

    #Save
    with open(json_path+file_name+'.json', 'w') as fp:
        json.dump(d, fp)

100%|██████████| 2/2 [00:29<00:00, 12.95s/it]
